In [1]:
#디비끼리 연결은 디비에서 쿼리문으로 실행

# 공개 문서 목록 수집 (문서ID + 제목만)
# pip install selenium pandas pymysql sqlalchemy openpyxl

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy import create_engine
from sqlalchemy.types import BIGINT, VARCHAR
import pandas as pd
import json
import time
import re
from pathlib import Path

# ─────────────────────────────────────────────────────────
# 설정
# ─────────────────────────────────────────────────────────
BASE = "http://office.anyfive.com/home/"

# DB 설정
DB_USER = "root"
DB_PASS = "1234"
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "any_approval" #컨트롤러 - 통합 DB명

# 기간 설정
# 5년 이하 경고창떠서 두번 쪼개서 실행함
END_YEAR = 2010  #컨트롤러 - 폴더 기간에 맞게 수정 (2025, 2020, 2015, 2010)
# START_DATE = "2011-01-01"  #컨트롤러 - 시작일
# END_DATE = "2013-12-31"  #컨트롤러 - 종료일

START_DATE = "2010-01-01"  #컨트롤러 - 시작일
END_DATE = "2010-12-31"  #컨트롤러 - 종료일

# ─────────────────────────────────────────────────────────
# 유틸 함수
# ─────────────────────────────────────────────────────────
def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    
    # 최상위
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    
    # 프레임 내부
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
    
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css, max_retries=3, wait_time=2):
    """selector_css 요소가 보이는 프레임으로 전환"""
    for attempt in range(max_retries):
        driver.switch_to.default_content()
        time.sleep(wait_time)
        
        # 최상위에서 찾기
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            print(f"✓ 요소 발견 (최상위)")
            return True
        
        # 프레임 내부 탐색
        frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
        for idx, fr in enumerate(frames):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(fr)
                time.sleep(0.3)
                
                if driver.find_elements(By.CSS_SELECTOR, selector_css):
                    print(f"✓ 요소 발견 (프레임 {idx})")
                    return True
            except:
                continue
        
        driver.switch_to.default_content()
        
        if attempt < max_retries - 1:
            print(f"⚠️ 요소 미발견, 재시도 중... ({attempt+1}/{max_retries})")
    
    return False

def set_kendo_dropdown_by_text(wrapper_css, item_text, sec=15):
    """Kendo DropDownList wrapper 클릭 → 텍스트로 선택"""
    w = wait_click_css(wrapper_css, sec)
    listbox_id = w.get_attribute("aria-owns")
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", w)
    driver.execute_script("arguments[0].click();", w)
    time.sleep(0.5)
    xp = f"//ul[@id='{listbox_id}']//li[contains(normalize-space(.), '{item_text}')]"
    el = WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.XPATH, xp)))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    driver.execute_script("arguments[0].click();", el)

def set_input_value(css, value):
    """입력 필드에 값 설정"""
    el = wait_css(css, 10)
    driver.execute_script(
        "arguments[0].value = arguments[1];"
        "arguments[0].dispatchEvent(new Event('input',{bubbles:true}));"
        "arguments[0].dispatchEvent(new Event('change',{bubbles:true}));",
        el, value
    )
    return el

def norm_space(s):
    """공백 정규화"""
    if not s:
        return ""
    s = s.replace('\xa0', ' ')
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

# ─────────────────────────────────────────────────────────
# 1) 브라우저 시작
# ─────────────────────────────────────────────────────────
print("="*70)
print("🚀 공개 문서 수집 시작")
print(f"📅 기간: {START_DATE} ~ {END_DATE} (endYear={END_YEAR})")
print("="*70)

opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-popup-blocking")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(), options=opts)

driver.get(BASE)
time.sleep(1.2)

# ─────────────────────────────────────────────────────────
# 2) 로그인 대기
# ─────────────────────────────────────────────────────────
input("✋ 로그인 완료 후 Enter를 눌러주세요... ")

# ─────────────────────────────────────────────────────────
# 3) 전자결재 → 결재 문서관리
# ─────────────────────────────────────────────────────────
print("\n📂 전자결재 메뉴 진입 중...")
click_in_all_frames(
    css_list=["a[href='/apr/'].left_menu", "a[href='/apr/']"],
    xp_list=["//a[@href='/apr/']"],
    text_contains="전자결재"
)
time.sleep(0.8)

print("📂 결재 문서관리 진입 중...")
click_in_all_frames(
    css_list=["li[onclick*='managementDoc']"],
    xp_list=["//*[@onclick and contains(.,'managementDoc')] | //*[contains(.,'결재 문서관리')]"],
    text_contains="결재 문서관리"
)
time.sleep(1.0)

# ─────────────────────────────────────────────────────────
# 4) 필터 설정
# ─────────────────────────────────────────────────────────
print("\n⚙️ 필터 설정 중...")
switch_into_frame_having("header, .tabConts_headerBx, #searchFrm, form#managementdocForm")

# 상태 = 완료
STATUS_WRAPPER = "span.k-dropdown[aria-owns='documentStatusFilter_listbox']"
try:
    print("  - 상태: 완료")
    set_kendo_dropdown_by_text(STATUS_WRAPPER, "완료")
except Exception:
    sel = driver.find_elements(By.CSS_SELECTOR, "#documentStatusFilter")
    if sel:
        driver.execute_script("""
            const sel=arguments[0];
            for (const o of sel.options){
                if (o.text.includes('완료')){
                    sel.value=o.value;
                    sel.dispatchEvent(new Event('change',{bubbles:true}));
                    break;
                }
            }
        """, sel[0])

# 📌 문서공개 = 공개
OPEN_WRAPPER = "span.k-dropdown[aria-owns='documentOpenFilter_listbox']"
try:
    print("  - 문서공개: 공개")
    set_kendo_dropdown_by_text(OPEN_WRAPPER, "공개")
except Exception as e:
    print(f"    ⚠️ 문서공개 설정 실패: {e}")
    # fallback: select 태그 직접 조작
    sel = driver.find_elements(By.CSS_SELECTOR, "#documentOpenFilter")
    if sel:
        driver.execute_script("""
            const sel=arguments[0];
            for (const o of sel.options){
                if (o.text.includes('공개') && !o.text.includes('비공개')){
                    sel.value=o.value;
                    sel.dispatchEvent(new Event('change',{bubbles:true}));
                    break;
                }
            }
        """, sel[0])

# 시작일
try:
    print(f"  - 시작일: {START_DATE}")
    set_input_value("#startDatePicker", START_DATE)
except Exception:
    pass

# 종료일
try:
    print(f"  - 종료일: {END_DATE}")
    set_input_value("#endDatePicker", END_DATE)
except Exception:
    dp_inputs = driver.find_elements(By.CSS_SELECTOR, "span.k-widget.k-datepicker input.k-input")
    if len(dp_inputs) >= 2:
        driver.execute_script(
            "arguments[0].value = arguments[1];"
            "arguments[0].dispatchEvent(new Event('input',{bubbles:true}));"
            "arguments[0].dispatchEvent(new Event('change',{bubbles:true}));",
            dp_inputs[1], END_DATE
        )

# 조회 버튼 클릭
print("\n🔍 조회 버튼 클릭...")
try:
    btn = wait_click_css("#documentSearch")
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
except Exception:
    click_in_all_frames(
        xp_list=["//button[normalize-space()='조회']",
                 "//a[normalize-space()='조회']"],
        text_contains="조회"
    )
time.sleep(2)

# ─────────────────────────────────────────────────────────
# 5) 그리드에서 데이터 수집
# ─────────────────────────────────────────────────────────
print("\n📊 그리드 데이터 수집 중...")

GRID = "table[role='grid']"

if not switch_into_frame_having(GRID):
    print("❌ 그리드를 찾을 수 없습니다!")
    driver.quit()
    exit(1)

# 로딩 대기
try:
    WebDriverWait(driver, 15).until(
        EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
    )
except:
    pass

time.sleep(1)

# 그리드 파싱
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, GRID + " tbody tr"))
)

table = driver.find_element(By.CSS_SELECTOR, GRID)
rows = table.find_elements(By.CSS_SELECTOR, "tbody tr[role='row'], tbody tr")

# 헤더 분석 (문서ID, 제목 컬럼 위치 찾기)
th_all = table.find_elements(By.CSS_SELECTOR, "thead th")
abs_index_by_field = {}
for i, th in enumerate(th_all):
    field = (th.get_attribute("data-field") or "").strip()
    if field:
        abs_index_by_field[field] = i

def td_text_by_field(tr, field):
    """tr에서 특정 field의 텍스트 추출"""
    tds = tr.find_elements(By.CSS_SELECTOR, "td")
    idx = abs_index_by_field.get(field)
    if idx is not None and idx < len(tds):
        raw = tds[idx].get_attribute("textContent")
        if raw:
            return norm_space(raw)
        raw = tds[idx].text
        if raw:
            return norm_space(raw)
    
    # fallback: data-field 속성으로 직접 찾기
    els = tr.find_elements(By.CSS_SELECTOR, f"td[data-field='{field}']")
    if els:
        raw = els[0].get_attribute("textContent")
        if raw:
            return norm_space(raw)
        raw = els[0].text
        if raw:
            return norm_space(raw)
    
    # fallback: 고정 인덱스 (양식명=1, 제목=3 등)
    field_map = {
        "form_nm": 1,
        "appr_doc_tit": 3,
    }
    fallback_idx = field_map.get(field)
    if fallback_idx is not None and fallback_idx < len(tds):
        raw = tds[fallback_idx].get_attribute("textContent")
        if raw:
            return norm_space(raw)
        raw = tds[fallback_idx].text
        if raw:
            return norm_space(raw)
    
    return ""

# 데이터 수집
results = []
for tr in rows:
    tds = tr.find_elements(By.CSS_SELECTOR, "td")
    if not tds:
        continue
    
    # 문서ID (체크박스 value에서)
    cb = tr.find_elements(By.CSS_SELECTOR, "td input[name='aprDoc_grid_ckbx']")
    doc_id = cb[0].get_attribute("value").strip() if cb else ""
    
    # 제목
    title_from_cb = norm_space(cb[0].get_attribute("data-title")) if cb else ""
    title = title_from_cb or td_text_by_field(tr, "appr_doc_tit")
    
    if doc_id and title:
        results.append({
            "문서ID": doc_id,
            "title": title,
            "end_year": END_YEAR
        })

print(f"✅ 수집 완료: {len(results)}건")

# ─────────────────────────────────────────────────────────
# 6) JSON 저장
# ─────────────────────────────────────────────────────────
json_path = Path("one_isPublicCrawling_result.json") #컨트롤러
json_path.write_text(
    json.dumps(results, ensure_ascii=False, indent=2),
    encoding="utf-8"
)
print(f"\n💾 JSON 저장: {json_path} ({len(results)}건)")

# ─────────────────────────────────────────────────────────
# 7) MariaDB 저장
# ─────────────────────────────────────────────────────────
print("\n💾 MariaDB 저장 중...")

try:
    engine = create_engine(
        f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4"
    )
    print(f"✅ DB 연결: {DB_HOST}:{DB_PORT}/{DB_NAME}")
    
    df = pd.DataFrame(results)
    
    TABLE_NAME = "public" #컨트롤러 - 통합 테이블명
    
    df.to_sql(
        name=TABLE_NAME,
        con=engine,
        if_exists="append",  # replace → append로 변경
        index=False,
        dtype={
            "문서ID": BIGINT(),
            "title": VARCHAR(500),
            "end_year": BIGINT()
        }
    )
    
    print(f"✅ DB 저장 완료: {TABLE_NAME} 테이블 ({len(df)}건)")
    
except Exception as e:
    print(f"❌ DB 저장 실패: {e}")

# ─────────────────────────────────────────────────────────
# 8) 완료
# ─────────────────────────────────────────────────────────
print("\n" + "="*70)
print("🎉 수집 완료!")
print("="*70)
print(f"📊 총 {len(results)}건 수집")
print(f"📁 저장 위치:")
print(f"   - JSON: one_isPublicCrawling_result.json")
print(f"   - DB: {DB_NAME}.{TABLE_NAME} (endYear={END_YEAR})")
print("="*70)

# 결과 미리보기
if results:
    print("\n📋 수집 데이터 미리보기 (처음 5건):")
    for i, item in enumerate(results[:5], 1):
        print(f"  {i}. 문서ID: {item['문서ID']}, 제목: {item['title'][:50]}..., endYear: {item['end_year']}")

driver.quit()

🚀 공개 문서 수집 시작
📅 기간: 2010-01-01 ~ 2010-12-31 (endYear=2010)


✋ 로그인 완료 후 Enter를 눌러주세요...  



📂 전자결재 메뉴 진입 중...
📂 결재 문서관리 진입 중...

⚙️ 필터 설정 중...
✓ 요소 발견 (프레임 1)
  - 상태: 완료
  - 문서공개: 공개
  - 시작일: 2010-01-01
  - 종료일: 2010-12-31

🔍 조회 버튼 클릭...

📊 그리드 데이터 수집 중...
✓ 요소 발견 (프레임 1)
✅ 수집 완료: 31건

💾 JSON 저장: one_isPublicCrawling_result.json (31건)

💾 MariaDB 저장 중...
✅ DB 연결: 127.0.0.1:3306/any_approval
✅ DB 저장 완료: public 테이블 (31건)

🎉 수집 완료!
📊 총 31건 수집
📁 저장 위치:
   - JSON: one_isPublicCrawling_result.json
   - DB: any_approval.public (endYear=2010)

📋 수집 데이터 미리보기 (처음 5건):
  1. 문서ID: 2002816, 제목: [계약품의]R&D특허센터 전자연구노트 활성화를 위한 홈페이지 구축 건..., endYear: 2010
  2. 문서ID: 2002722, 제목: 국내출장비 신청 품의 (외주 김현주 과장)..., endYear: 2010
  3. 문서ID: 2002701, 제목: [품의] 연차사용품의 (2010.10.18 월 - 오전 반차)..., endYear: 2010
  4. 문서ID: 2002700, 제목: 국내출장 비용 청구서..., endYear: 2010
  5. 문서ID: 2002693, 제목: [LG화학/기술연구원] 서울 트윈 타워 출장 신청서..., endYear: 2010
